<a href="https://colab.research.google.com/github/renyuanL/110_project/blob/main/_ry_Confidence_Interval_and_Hypothesis_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Financial Python
## Confidence Interval and Hypothesis Testing

- Ref: https://www.quantconnect.com/tutorials/introduction-to-financial-python/confidence-interval-and-hypothesis-testing#Confidence-Interval-and-Hypothesis-Testing-Introduction


In [ ]:
# https://www.quantconnect.com/tutorials/introduction-to-financial-python/confidence-interval-and-hypothesis-testing#Confidence-Interval-and-Hypothesis-Testing-Introduction


In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

'''
qb = QuantBook()
spy = qb.AddEquity("SPY").Symbol
'''
#get SPY data from August 2010 to the present
'''
start_date = datetime(2010, 8, 1, 0, 0, 0)
end_date = qb.Time
spy_table = qb.History(spy, start_date, end_date, Resolution.Daily)

spy_total = spy_table[['open','close']]
'''
import pandas_datareader as pdr

spy_table= pdr.get_data_yahoo('SPY',start='2010-08')
spy_table
spy_total= spy_table[['Open','Close']]
spy_total.columns= ['open','close']


In [10]:



#calculate log returns
spy_log_return = np.log(spy_total.close).diff().dropna()

print('Population mean:', np.mean(spy_log_return))
#[out]: Population mean: 0.000443353825615
print('Population standard deviation:',np.std(spy_log_return))
#[out]: Population standard deviation: 0.00784267293815


Population mean: 0.00048535249884885
Population standard deviation: 0.01075169962667365


In [11]:
print('10 days sample returns:', np.mean(spy_log_return.tail(10)))
#[out]: 10 days sample returns: 0.000845189915474
print('10 days sample standard deviation:', np.std(spy_log_return.tail(10)))
#[out]: 10 days sample standard deviation: 0.00313558001122
print('1000 days sample returns:', np.mean(spy_log_return.tail(1000)))
#[out]: 1000 days sample returns: 0.000462827047221
print('1000 days sample standard deviation:', np.std(spy_log_return.tail(1000)))
#[out]: 1000 days sample standard deviation: 0.00766589174299

10 days sample returns: 0.0012350804361541457
10 days sample standard deviation: 0.003422828669076326
1000 days sample returns: 0.0005602979943605533
1000 days sample standard deviation: 0.013036113231790701


In [12]:
#apply the formula above to calculate confidence interval
bottom_1 = np.mean(spy_log_return.tail(10))-1.96*np.std(spy_log_return.tail(10))/(np.sqrt(len((spy_log_return.tail(10)))))
upper_1 = np.mean(spy_log_return.tail(10))+1.96*np.std(spy_log_return.tail(10))/(np.sqrt(len((spy_log_return.tail(10)))))
bottom_2 = np.mean(spy_log_return.tail(1000))-1.96*np.std(spy_log_return.tail(1000))/(np.sqrt(len((spy_log_return.tail(1000)))))
upper_2 = np.mean(spy_log_return.tail(1000))+1.96*np.std(spy_log_return.tail(1000))/(np.sqrt(len((spy_log_return.tail(1000)))))
#print the outcomes
print('10 days 95% confidence inverval:', (bottom_1,upper_1))
#[out]: 10 days 95% confidence inverval: (-0.0010982627102681939, 0.002788642541217079)
print('1000 days 95% confidence inverval:', (bottom_2,upper_2))
#[out]: 1000 days 95% confidence inverval: (-1.230984558013321e-05, 0.00093796394002165957)


10 days 95% confidence inverval: (-0.0008864107522674248, 0.0033565716245757163)
1000 days 95% confidence inverval: (-0.00024768867474646284, 0.0013682846634675695)


In [13]:
mean_1000 = np.mean(spy_log_return.tail(1000))
std_1000 = np.std(spy_log_return.tail(1000))
mean_10 = np.mean(spy_log_return.tail(10))
std_10 = np.std(spy_log_return.tail(10))
s = pd.Series([mean_10,std_10,mean_1000,std_1000],index = ['mean_10', 'std_10','mean_1000','std_1000'])
print(s)

'''
[out]: mean_10      0.000845
       std_10       0.003136
       mean_1000    0.000463
       std_1000     0.007666
'''


mean_10      0.001235
std_10       0.003423
mean_1000    0.000560
std_1000     0.013036
dtype: float64


'\n[out]: mean_10      0.000845\n       std_10       0.003136\n       mean_1000    0.000463\n       std_1000     0.007666\n'

In [14]:
bottom = 0 - 1.64*std_1000/np.sqrt(1000)
upper = 0 + 1.64*std_1000/np.sqrt(1000)
print((bottom, upper))

#[out]: (-0.00039756352254768874, 0.00039756352254768874)


(-0.0006760704782324012, 0.0006760704782324012)


In [15]:
bottom = 0 - 1.96*std_1000/np.sqrt(1000)
upper = 0 + 1.96*std_1000/np.sqrt(1000)
print((bottom, upper))

#[out]: (-0.00047513689280089639, 0.00047513689280089639)


(-0.0008079866691070161, 0.0008079866691070161)


In [16]:
print(np.sqrt(1000)*(mean_1000 - 0)/std_1000)

#[out]: 1.90922032428


1.3591611234878334


In [17]:
import scipy.stats as st
print((1 - st.norm.cdf(1.9488)))

#[out]: 0.025659656888


0.02565965688799665


In [18]:
mean_1200 = np.mean(spy_log_return.tail(1200))
std_1200 = np.std(spy_log_return.tail(1200))
z_score = np.sqrt(1200)*(mean_1200 - 0)/std_1200
print('z-score = ',z_score)
#[out]: z-score =  2.19793023185
p_value = (1 - st.norm.cdf(z_score))
print('p_value = ',p_value)
#[out]: p_value =  0.0139770390655


z-score =  1.582371281814201
p_value =  0.05678241835411302


# Summary

- In this chapter we introduced confidence interval, especially that for the normal distribution, and hypothesis test. Now we know how to test our idea rigorously. Normal distribution and it's confidence interval can be applied to many quantitative finance theories, we will see it frequently in our following tutorials.

# Ref:
- 
https://www.quantconnect.com/tutorials/introduction-to-financial-python/confidence-interval-and-hypothesis-testing#Confidence-Interval-and-Hypothesis-Testing-Introduction
